In [25]:
import os
import random as rnd

import numpy as np
import pandas as pd 
import pgutils as pg

In [26]:
root_dir = r'C:\Users\eleon\PycharmProjects\SM_Exam\data\yes_small'
#root_dir = './data/yes_small'

In [27]:
test_data=[]
with open(os.path.join(root_dir, 'test.txt')) as f:
#with open(root_dir + "\\test.txt", "r") as f:

    test_data=f.readlines()
    
test_dataset = pg.data_to_list(test_data[2:])


In [4]:
train_data=[]
with open(os.path.join(root_dir,"train.txt"), "r") as f:
    train_data=f.readlines()
    
train_dataset = pg.data_to_list(train_data[2:])

NameError: name 'pg' is not defined

In [5]:
song_hash = pd.read_csv(os.path.join(root_dir,"song_hash.txt"), sep="\t", header=None)


In [ ]:
U_old = np.random.rand(S,d)
U_new = np.empty_like(U_old)
V_old = np.random.rand(S,d)
V_new = np.empty_like(V_old)
#tau predefined learning rate
tau = 0.5
#number of transitions in the training set
N=np.sum(T)

#try for 100-200 iterations
for k in range(200):
    
    D2_mat = delta2(U_old,V_old)
    Zr_vec = zeta2_r(a,U_old, V_old, D2_mat,C_r)
    dif_mat = difMat(U_old, V_old, S, d)
    
    #print(Z2_vec)
    
    updateU(S, Zr_vec, D2_mat, T, U_new, U_old, V_old, tau, N, dif_mat)
    updateV(S, Zr_vec, D2_mat, T,U_old, V_new, V_old, tau, N, dif_mat)
    
    U_new, U_old = U_old, U_new
    V_new, V_old = V_old, V_new
    if k==10:
        landmarkGenerator(S)
#empirically update landmarks every 10 iterations
#A iteration means a full pass on the training dataset.
#fix the landmarks after 100 iteration to ensure convergence   

In [17]:
# Compute and print the log-likelihood
#We evaluate test performance using the average log-likelihood as our metric. 
#It is deﬁned as log(Pr(Dtest))/Ntest, 
#where Ntest is the number of transitions in test set


In [38]:
#setto i parametri
d=5;
S=max([])
l=rnd.choice([0.0001, 0.001, 0.01, 0.1, 1, 10, 20, 50, 100, 500, 1000])
ni=rnd.choice([0,l])
r=8
n_landmarks=50
n_iters=rnd.choice([100,200])
tollerance=#da definire

def EMC(song_hash,train_dataset,r,n_landmarks):
    #1,definisco le funzioni di inizializzazione
    S=len(song_hash[0].values)
    U_old=np.random.rand(S,d)
    V_old=np.random.rand(S,d)
    U_new=np.empty_like(U_old)
    V_new=np.empty_like(V_old)
    T=pg.transition_count(S,train_dataset)
    C=landmarkInizialization(r,n_landmarks,S,T)
    C_rs=landmarkThresholdExtention(S,r,n_landmarks,C)    
    
    for iter in range(n_iters):
        #2,update dei landmarks
        if (iter%10==0 & iter<100):
            C=updateLandmarks(r,S,C)
        #3,Update U e V (new or old?)
        U_old=update(U_old)
        V_old=update(V_old)
        #4,criterio d'arresto
        if (stopCriteria(U_new,U_old)>tollerance | stopCriteria(V_new,V_old)>tollerance):
            break
        #5,swap matrix
        U_new, U_old = U_old, U_new
        V_new, V_old = V_old, V_new
